In [1]:
import numpy as np
import pandas as pd
import os

## 資料探索與前處理

In [2]:
# 訓練資料的目錄
projDir = "/kaggle/input/home-credit-credit-risk-model-stability/"
trainDir = os.path.join(projDir, "csv_files/train/")

In [3]:
# 讀取訓練數據

# base.csv
path = os.path.join(trainDir, 'train_base.csv')
train_base = pd.read_csv(path)

# static(depth=0)
# train_static is divided into two parts, 0 and 1. read and concat one by one
path = os.path.join(trainDir, 'train_static_0_0.csv')
train_static0 = pd.read_csv(path)
path = os.path.join(trainDir, 'train_static_0_1.csv')
train_static1 = pd.read_csv(path)
train_static = pd.concat([train_static0, train_static1], ignore_index=True)
del train_static0, train_static1

# static_cb
path = os.path.join(trainDir, 'train_static_cb_0.csv')
train_static_cb = pd.read_csv(path)

#------------------------------------------------------------------------------------#

# Join Table
train_data = pd.merge(train_base, train_static, how="left", on="case_id")
train_data = pd.merge(train_data, train_static_cb, how="left", on="case_id")
del train_base, train_static, train_static_cb

#------------------------------------------------------------------------------------#

# 刪除非數字的column
for col in train_data.columns:
    if not (train_data[col].dtype in (np.int64, np.float64)):
        train_data = train_data.drop(col, axis=1)

/tmp/ipykernel_18/1472793701.py:10: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static0 = pd.read_csv(path)
/tmp/ipykernel_18/1472793701.py:12: DtypeWarning: Columns (20,45,46,56,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static1 = pd.read_csv(path)
/tmp/ipykernel_18/1472793701.py:18: DtypeWarning: Columns (1,2,3,4,7,45,46,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  train_static_cb = pd.read_csv(path)


In [4]:
# 將目標變量和特徵變量分開
X = train_data.drop(columns=['target'])
y = train_data['target']

# 將數據劃分為訓練集和測試集
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
del X
del y

## 數據分析方法與模型選擇

### Undersampling

In [5]:
# 欠採樣: 減少target=0的data

from sklearn.utils import resample

# 將 DataFrame 轉換為 NumPy 陣列
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

# 隨機抽取與目標樣本數量相同的非目標樣本
X_train_0, y_train_0 = resample(X_train[y_train == 0], y_train[y_train == 0], replace=False, n_samples=np.sum(y_train == 1))
X_test_0, y_test_0 = resample(X_test[y_test == 0], y_test[y_test == 0], replace=False, n_samples=np.sum(y_test == 1))

# 合併目標樣本和非目標樣本
X_train = np.concatenate((X_train_0, X_train[y_train == 1]), axis=0)
y_train = np.concatenate((y_train_0, y_train[y_train == 1]), axis=0)
X_test = np.concatenate((X_test_0, X_test[y_test == 1]), axis=0)
y_test = np.concatenate((y_test_0, y_test[y_test == 1]), axis=0)

# 檢查欠採樣後的資料量
print("欠採樣後的訓練資料量：", len(X_train))
print("欠採樣後的測試資料量：", len(X_test))

# 檢查欠採樣後的目標資料分佈
unique, counts = np.unique(y_train, return_counts=True)
print("欠採樣後的訓練目標資料分佈：", dict(zip(unique, counts)))
unique, counts = np.unique(y_test, return_counts=True)
print("欠採樣後的測試目標資料分佈：", dict(zip(unique, counts)))

欠採樣後的訓練資料量： 76882
欠採樣後的測試資料量： 19106
欠採樣後的訓練目標資料分佈： {0: 38441, 1: 38441}
欠採樣後的測試目標資料分佈： {0: 9553, 1: 9553}


### Model1. 梯度提升樹（Gradient Boosting Decision Tree，GBDT）
##### 優點：
##### 1. 高效性： LightGBM 是一種高效的梯度提升樹模型，具有優秀的訓練速度和預測速度。它使用了基於直方圖的方法來加速訓練過程，並且通常比其他梯度提升樹庫（如XGBoost）更快。
##### 2. 低內存使用： LightGBM 使用了一種稱為GOSS（Gradient-based One-Side Sampling）的方法，在訓練過程中減少了內存使用。這使得它可以處理大型數據集而不會出現內存不足的問題。
##### 3. 高準確性： LightGBM 在處理大型和高維數據集時通常表現出色，並且提供了很好的預測性能。
##### 4. 支持並行處理： LightGBM 支持並行處理，可以在多核 CPU 上進行訓練，從而加速了訓練過程。
##### 5. 自動處理類別特徵： LightGBM 可以自動處理類別特徵，無需額外的編碼。
##### 缺點：
##### 1. 對參數敏感： 與其他梯度提升樹模型一樣，LightGBM 的性能取決於參數的調整，對參數較為敏感，需要一些經驗和時間來進行調參。
##### 2. 需要較多數據： LightGBM 在處理小型數據集時可能不如其他模型表現好，因為它的高效性主要體現在處理大型數據集時

In [6]:
import lightgbm as lgb

# 創建 LightGBM 的訓練資料集
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test, reference=lgb_train)
del X_train
del y_train

# 設置參數
params = {
    'boosting_type': 'gbdt',         # 使用梯度提升樹
    'objective': 'binary',           # 二元分類任務
    'metric': 'binary_error',        # 評估指標為二元分類錯誤率
    'num_leaves': 31,                # 每棵樹的最大葉子數量
    'learning_rate': 0.05,           # 學習速率
    'feature_fraction': 0.9,         # 特徵抽樣比例
    'bagging_fraction': 0.8,         # 樣本抽樣比例
    'bagging_freq': 5,               # 樣本抽樣的頻率
    'verbose': 0,                    # 顯示訓練信息
    'early_stopping_rounds': 10
}

# 訓練模型
num_round = 100                      # 迭代輪次
bst = lgb.train(params, lgb_train, num_round, valid_sets=[lgb_train, lgb_test])

# 預測
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

### Model2. RNN
##### RNN的優點：
##### 1. 能夠處理序列數據，保留先前時間步的信息。
##### 2. 模型相對較簡單，易於理解和實現。
##### RNN的缺點：
##### 3. 容易出現梯度消失或梯度爆炸的問題，尤其在處理長序列數據時效果不佳。
##### 4. 難以捕捉長期依賴關係

In [7]:
# def focal_loss(y_true, y_pred, alpha=0.25, gamma=2.0):
#     # Calculate cross entropy loss
#     ce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred, from_logits=False)
    
#     # Calculate focal weights
#     p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
#     focal_weight = alpha * tf.pow(1 - p_t, gamma)
    
#     # Apply focal weights to cross entropy loss
#     focal_loss = focal_weight * ce_loss
    
#     # Return the mean loss over the batch
#     return tf.reduce_mean(focal_loss)

In [8]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.optimizers import AdamW
# # from sklearn.utils.class_weight import compute_class_weight

# # 將資料轉換成3D形狀
# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# # 創建 Sequential 模型
# model = Sequential()

# # 添加第一層 SimpleRNN 層
# model.add(SimpleRNN(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
# model.add(Dropout(0.3))  # 添加 dropout 避免過擬合

# # 添加第二層 SimpleRNN 層
# model.add(SimpleRNN(50, return_sequences=True, activation='relu'))
# model.add(Dropout(0.3))

# # 添加第三層 SimpleRNN 層
# model.add(SimpleRNN(50, activation='relu'))
# model.add(Dropout(0.3))

# # 添加輸出層
# model.add(Dense(1, activation='sigmoid'))

# # 定義 AdamW 優化器，設置初始學習率和權重衰減
# # 嘗試不同的優化器 ：AdamW是Adam的一個變種，引入了一個額外的權重衰減（Weight Decay）項目，有助於減小參數的數值大小，從而減少過擬合的風險。
# # 調整學習率（Learning Rate）：嘗試不同的學習率值，有時降低學習率可以幫助模型更好地收斂，特別是在資料不平衡的情況下。
# optimizer = AdamW(learning_rate=0.001, weight_decay=1e-4)

# # 編譯模型
# # focal_loss 損失函數，可以自動為少數類別分配更大的權重
# model.compile(optimizer=optimizer, loss=focal_loss, metrics=['accuracy'])

# # 定義 EarlyStopping
# # 調整Early Stopping的參數：嘗試不同的patience值，以及monitor參數，這可能會影響到模型的停止訓練的時機。
# early_stopping = EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)

# # 設置類別加權，例如給予類別0權重1，給予類別1權重100
# # 1. 使用 np.unique(y_train) 函數獲取訓練數據中所有類別的標籤。
# # 2. 使用 compute_class_weight 函數計算每個類別的權重，參數 'balanced' 指定使用平衡策略。
# # 3. 將計算出的類別權重存儲在 class_weight 字典中
# # class_weight = compute_class_weight('balanced', np.unique(y_train), y_train)
# class_weight = {0: 1, 1: 100}

# # 訓練模型
# # 批量大小: 嘗試減小批量大小，可以讓模型在每個更新步驟中看到更多樣化的樣本，可能有助於緩解數據不平衡問題。
# history = model.fit(X_train, y_train, epochs=3, batch_size=128, validation_data=(X_test, y_test), callbacks=[early_stopping], class_weight=class_weight) # class_weight='balanced' not work

# # 評估
# model.evaluate(X_test, y_test)

# # 預測
# y_pred = model.predict(X_test)
# y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

### Model3. LSTM
##### LSTM的優點：
##### 1. 能夠有效地解決梯度消失或梯度爆炸的問題，能夠處理長序列數據。
##### 2. 能夠捕捉長期依賴關係，適用於需要考慮長期記憶的任務。
##### LSTM的缺點：
##### 1. 模型相對複雜，訓練時間較長。
##### 2. 需要調參和優化，有時可能會出現過擬合的問題。

In [9]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping

# # 將 DataFrame 轉換為 NumPy 陣列
# X_train = X_train.to_numpy()
# X_test = X_test.to_numpy()

# # 將資料轉換成3D形狀
# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# # 創建 Sequential 模型
# model = Sequential()

# # 添加第一層 LSTM 層
# model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
# model.add(Dropout(0.3))  # 添加 dropout 避免過擬合

# # 添加第二層 LSTM 層
# model.add(LSTM(50, return_sequences=True, activation='relu'))
# model.add(Dropout(0.3))  # 添加 dropout 避免過擬合

# # 添加第三層 LSTM 層
# model.add(LSTM(50, activation='relu'))
# model.add(Dropout(0.3))  # 添加 dropout 避免過擬合

# # 添加輸出層
# model.add(Dense(1, activation='sigmoid'))

# # 編譯模型
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # 定義 EarlyStopping
# early_stopping = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)

# # 設置類別加權，例如給予類別0權重1，給予類別1權重100
# class_weight = {0: 1, 1: 100}

# # 訓練模型
# history = model.fit(X_train, y_train, epochs=1, batch_size=2048, validation_data=(X_test, y_test), callbacks=[early_stopping], class_weight=class_weight)

# # 預測
# y_pred = model.predict(X_test)
# y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

## 評估模型

In [10]:
from sklearn.metrics import accuracy_score, classification_report

# 評估模型
accuracy = accuracy_score(y_test, y_pred_binary)
print("準確率:", accuracy)

# 顯示分類報告
print("分類報告:")
print(classification_report(y_test, y_pred_binary))

準確率: 0.7210823824976447
分類報告:
              precision    recall  f1-score   support

           0       0.73      0.70      0.71      9553
           1       0.71      0.74      0.73      9553

    accuracy                           0.72     19106
   macro avg       0.72      0.72      0.72     19106
weighted avg       0.72      0.72      0.72     19106



## Submission

In [11]:
# 訓練資料的目錄
testDir = os.path.join(projDir, "csv_files/test/")

# 讀取訓練數據

# base.csv
path = os.path.join(testDir, 'test_base.csv')
test_base = pd.read_csv(path)

# static(depth=0)
# test_static is divided into three parts: 0, 1, and 2.
path = os.path.join(testDir, 'test_static_0_0.csv')
test_static0 = pd.read_csv(path)
path = os.path.join(testDir, 'test_static_0_1.csv')
test_static1 = pd.read_csv(path)
path = os.path.join(testDir, 'test_static_0_2.csv')
test_static2 = pd.read_csv(path)
test_static = pd.concat([test_static0, test_static1, test_static2], ignore_index=True)
del test_static0, test_static1, test_static2

# static_cb
path = os.path.join(testDir, 'test_static_cb_0.csv')
test_static_cb = pd.read_csv(path)

# Join Table
X_test = pd.merge(test_base, test_static, how="left", on="case_id")
X_test = pd.merge(X_test, test_static_cb, how="left", on="case_id")
del test_base, test_static, test_static_cb

# 刪非數col
for col in X_test.columns:
    if not (X_test[col].dtype in (np.int64, np.float64)):
        X_test = X_test.drop(col, axis=1)

In [12]:
# LGBM
y_test_pred = bst.predict(X_test, num_iteration=bst.best_iteration, predict_disable_shape_check=True)

# # RNN, LSTM
# X_test_array = X_test.to_numpy()
# X_test_rnn = X_test_array.reshape(X_test_array.shape[0], X_test_array.shape[1], 1)
# y_test_pred = model.predict(X_test_rnn)

# 轉成0或1
y_test_pred_binary = [1 if pred > 0.5 else 0 for pred in y_test_pred]
y_test_pred_binary

[1, 1, 1, 1, 1, 0, 1, 0, 1, 0]

In [13]:
submission = pd.DataFrame({
    "case_id": X_test["case_id"].to_numpy(),
    "score": y_test_pred_binary
}).set_index('case_id')
submission.to_csv("./submission.csv")

pd.read_csv("./submission.csv")

,case_id,score
0,57543,1
1,57549,1
2,57551,1
3,57552,1
4,57569,1
5,57630,0
6,57631,1
7,57632,0
8,57633,1
9,57634,0
